In [8]:
!pip install selenium webdriver_manager pandas fake-useragent

  Using cached fake_useragent-2.2.0-py3-none-any.whl.metadata (17 kB)
Using cached fake_useragent-2.2.0-py3-none-any.whl (161 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import re
import time
import pandas as pd
import traceback
from fake_useragent import UserAgent

In [27]:
# Chrome 옵션 설정
# 유저 에어전트 객체 생성
ua = UserAgent()

# 크롬 드라이버 옵션 설정 (브라우저 창 없이 실행)
options = Options()
options.add_argument('--headless')
options.add_argument("--window-size=1920,1080")
# options.add_argument('--disable-gpu')
options.add_argument(f'user-agent={ua.random}')

# 타겟 URL
category_code = 104001 # 104001: 스킨케어 코드
url = f"https://www.musinsa.com/main/beauty/ranking?storeCode=beauty&sectionId=231&contentsId=&categoryCode={category_code}&gf=A"

In [29]:
# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # 페이지 로드
    driver.get(url)
    time.sleep(3)  # 초기 로딩 대기

    # 스크롤 다운 (동적 컨텐츠 로딩)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 새 컨텐츠 로딩 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 데이터 추출
    data = []
    base_xpath = "/html/body/div[1]/div/main/article/div[2]"
    
    i = 1
    while i < 2:
        try:
            item = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, f"{base_xpath}/div[{i}]"))
            )
            
            # 요소가 보일 때까지 스크롤
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", item)
            time.sleep(0.5)  # 렌더링 대기

            # product_id
            a_tag = driver.find_element(By.XPATH, f"{base_xpath}/div[{i}]/div[1]/a")
            href = a_tag.get_attribute("href")
            # 정규표현식으로 id 추출
            match = re.search(r'/products/(\d+)', href)
            if match:
                product_id = match.group(1)
                print(f"{i}: {product_id}")
            
            # 브랜드 요소 추출 (명시적 대기 추가)
            brand_element = WebDriverWait(item, 5).until(
                EC.visibility_of_element_located((By.XPATH, "./div[2]/div[1]/a[1]/p"))
            )
            brand = brand_element.text.strip()
            print(f"brand : {brand}")

            # sales
            try:
                sales_element = WebDriverWait(item, 5).until(
                    EC.visibility_of_element_located((By.XPATH, "./div[1]/div[1]/div/span/span"))
                )
                sales = brand_element.text.strip()
            except (TimeoutException, NoSuchElementException):
                sales = "0"
            print(f"sales : {sales}")
            
            # product_name
            product_name_element = WebDriverWait(item, 10).until(
                EC.visibility_of_element_located((By.XPATH, "./div[2]/div[1]/a[2]/p"))                
            )
            product_name = product_name_element.text.strip()
            print(f"product_name : {product_name}")
            
            # discount
            try:
                discount_element = WebDriverWait(item, 5).until(
                    EC.visibility_of_element_located((By.XPATH, "./div[1]/div[2]/div[1]/div/span[1]"))
                )
                discount = discount_element.text.strip()
            except (TimeoutException, NoSuchElementException):
                discount = "0"
            print(f"discount : {discount}")
            
            # price
            try:
                price_element = WebDriverWait(item, 5).until(
                    EC.visibility_of_element_located((By.XPATH, "./div[2]/div[1]/div/span[2]"))
                )
                price = price_element.text.strip()
            except (TimeoutException, NoSuchElementException):
                try:
                    price_element = WebDriverWait(item, 2).until(
                        EC.visibility_of_element_located((By.XPATH, "./div[2]/div[1]/div/span[1]"))
                    )
                    price = price_element.text.strip()
                except (TimeoutException, NoSuchElementException):
                    price = "0"
            print(f"price : {price}")
            
            # numbers_of_views 
            try:
                numbers_of_views_element = WebDriverWait(item, 5).until(
                    EC.visibility_of_element_located((By.XPATH, "./div[1]/div[2]/div[2]/div/span[1]"))
                )
                numbers_of_views = numbers_of_views_element.text.strip()
            except (TimeoutException, NoSuchElementException):
                numbers_of_views = "0"
            print(f"numbers_of_views : {numbers_of_views}")
            # 데이터 저장
            data.append(
                {
                    'product_id': product_id,
                    'sales': sales,
                    'brand': brand,
                    'product_name': product_name,
                    'discount': discount,
                    'price': price,
                    'numbers_of_views': numbers_of_views,
                }
            )
        except Exception as ex:
            print(ex)
            traceback.print_exc()
            break  # i 반복 종료
        i += 1

 
finally:
    driver.quit()  # 드라이버 종료

1: 5177930
brand : 라네즈
sales : 0
product_name : 깜자 콜라보 워터뱅크 젤크림 50ml 세트
discount : 0
price : 34,590원
numbers_of_views : 0


In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import re

# 크롬 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

# 드라이버 실행
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.musinsa.com/main/beauty/ranking?storeCode=beauty&sectionId=231&contentsId=&categoryCode=104001&gf=A"
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

# 스크롤 다운 (동적 로딩 대비)
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 데이터 추출
product_data = []
max_empty_rows = 5  # 연속 빈 줄이 나오면 종료
empty_row_count = 0
row = 2  # div[2]부터 시작

while True:
    found_in_row = False
    col = 1
    empty_col_count = 0

    while True:
        item_xpath = f"/html/body/div[1]/div/main/article/div[{row}]/div[{col}]"
        try:
            item = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.XPATH, item_xpath))
            )
            found_in_row = True
            empty_col_count = 0

            # product_id 추출
            try:
                a_tag = item.find_element(By.XPATH, "./div[1]/a")
                href = a_tag.get_attribute("href")
                match = re.search(r'/products/(\d+)', href)
                product_id = match.group(1) if match else ""
            except Exception:
                product_id = ""

            # 브랜드 추출
            try:
                brand_element = item.find_element(By.XPATH, "./div[2]/div[1]/a[1]/p")
                brand = brand_element.text.strip()
            except Exception:
                brand = ""

            # 제품명 추출
            try:
                product_name_element = item.find_element(By.XPATH, "./div[2]/div[1]/a[2]/p")
                product_name = product_name_element.text.strip()
            except Exception:
                product_name = ""

            # 할인율 추출
            try:
                discount_element = item.find_element(By.XPATH, "./div[2]/div[1]/div/span[1]")
                discount = discount_element.text.strip()
            except NoSuchElementException:
                discount = "0"

            # 가격 추출
            try:
                price_element = item.find_element(By.XPATH, "./div[2]/div[1]/div/span[2]")
                price = price_element.text.strip()
            except NoSuchElementException:
                try:
                    price_element = item.find_element(By.XPATH, "./div[2]/div[1]/div/span[1]")
                    price = price_element.text.strip()
                except NoSuchElementException:
                    price = "0"

            # 관심 유저수 추출
            try:
                interest_element = item.find_element(By.XPATH, "./div[2]/div[2]/div/span[1]")
                interest = interest_element.text.strip()
            except NoSuchElementException:
                interest = "0"

            # 구매 유저수 추출
            try:
                purchase_element = item.find_element(By.XPATH, "./div[2]/div[2]/div/span[2]")
                purchase = purchase_element.text.strip()
            except NoSuchElementException:
                purchase = "0"

            product_data.append({
                'product_id': product_id,
                'brand': brand,
                'product_name': product_name,
                'discount': discount,
                'price': price,
                'interest': interest,
                'purchase': purchase
            })

            print(f"Row {row} Col {col} - {product_name} 수집 완료")
            col += 1

        except TimeoutException:
            # 열이 비었으면 다음 열로, 연속 6번 비면 한 줄 끝
            empty_col_count += 1
            if empty_col_count >= 6:
                break
            else:
                col += 1

    if found_in_row:
        empty_row_count = 0
    else:
        empty_row_count += 1
        if empty_row_count >= max_empty_rows:
            print("더 이상 아이템이 없어 크롤링 종료")
            break

    row += 1

driver.quit()

# 결과 출력
print(f"총 수집된 제품 수: {len(product_data)}")
for product in product_data:
    print(product)


Row 2 Col 1 - 깜자 콜라보 워터뱅크 젤크림 50ml 세트 수집 완료
Row 2 Col 2 - 5in1 올데이 보습 올인원 100ml 수집 완료
Row 2 Col 3 - [2종세트] 다이브인 저분자 히알루론산 세럼 50ml+40ml 수집 완료
Row 2 Col 4 - (1+1)레드 블레미쉬 포 맨 진정 올인원 150mL 수집 완료
Row 2 Col 5 - 윤조에센스 90ml 단품기획세트 수집 완료
Row 2 Col 6 - 신선초 진정에이징 워터 세럼 30ml 수집 완료
Row 3 Col 1 - 세라마이드 아토 로션 564ml + 트러블 미스트 30ml 증정 수집 완료
Row 3 Col 2 - 아쿠아 오아시스 수분 젤크림 80ml 수집 완료
Row 3 Col 3 - 깜자 콜라보 워터뱅크 젤크림 50ml 세트 + 선스크린 50ml 세트 + 젤 크림 스킨 170ml 세트 수집 완료
Row 3 Col 4 - [콜라보] 시카 PDRN 흔적 클리어 세럼 35ml+패드70매 수집 완료
Row 3 Col 5 - 세파시카 장벽개선 손상케어 핑크 앰플 30ml 수집 완료
Row 3 Col 6 - 리얼 히알루로닉 블루 100앰플 100ml x 2개 (+블루앰플마스크 5매 증정) 수집 완료
Row 4 Col 1 - 소나무 진정 시카 앰플 대용량 50ml (+소나무 마스크 2매 증정) 수집 완료
Row 4 Col 2 - 소나무 진정 시카 로션 250ml 수집 완료
Row 4 Col 3 - 장벽튼튼 루틴세트(만능기초145ml + 판테놀55ml + 크림50ml) 수집 완료
Row 4 Col 4 - 어드벤스드 스네일 96 뮤신 파워 에센스 수집 완료
Row 4 Col 5 - 그린 LHA 모공 패드 클리어 터치 60매 수집 완료
Row 4 Col 6 - [2pack] 1025 독도 수분 크림 50ml(+독도 워터겔 마스크 1매 증정) 수집 완료
Row 5 Col 1 - 하이퍼 하이드로 세럼 100ml (대용량) 수집 완료
Row 5 Col 2 - 아쿠아 오아시스 토너 300ml+2

In [31]:
product_data

[{'product_id': '5177930',
  'brand': '라네즈',
  'product_name': '깜자 콜라보 워터뱅크 젤크림 50ml 세트',
  'discount': '18%',
  'price': '34,590원',
  'interest': '459명이 보는 중',
  'purchase': '0'},
 {'product_id': '5197452',
  'brand': '블랙몬스터',
  'product_name': '5in1 올데이 보습 올인원 100ml',
  'discount': '9%',
  'price': '10,900원',
  'interest': '68명이 보는 중',
  'purchase': '0'},
 {'product_id': '2959816',
  'brand': '토리든',
  'product_name': '[2종세트] 다이브인 저분자 히알루론산 세럼 50ml+40ml',
  'discount': '5%',
  'price': '39,900원',
  'interest': '34명이 보는 중',
  'purchase': '0'},
 {'product_id': '1729476',
  'brand': '닥터지',
  'product_name': '(1+1)레드 블레미쉬 포 맨 진정 올인원 150mL',
  'discount': '39%',
  'price': '39,000원',
  'interest': '0',
  'purchase': '0'},
 {'product_id': '3139694',
  'brand': '설화수',
  'product_name': '윤조에센스 90ml 단품기획세트',
  'discount': '2%',
  'price': '137,200원',
  'interest': '0',
  'purchase': '0'},
 {'product_id': '4572564',
  'brand': '라씨엘르',
  'product_name': '신선초 진정에이징 워터 세럼 30ml',
  'discount': '15%

In [33]:
import pandas as pd

# product_data 리스트를 DataFrame으로 변환
df = pd.DataFrame(product_data)

# DataFrame을 CSV 파일로 저장 (utf-8 인코딩)
df.to_csv('musinsa_beauty_ranking.csv', index=False, encoding='utf-8-sig')

In [ ]:
def build_api_url(self, page: int = 1, size: int = 40, section_id: str = "231", category_code: str = "104001") -> str:
    """API URL 생성"""
    # 올바른 sections API 엔드포인트 사용
    base_url = f"https://api.musinsa.com/api2/hm/web/v5/pans/ranking/sections/{section_id}"

    params = {
        'storeCode': 'beauty',
        'categoryCode': category_code,
        'contentsId': '',
        'gf': 'A',  # 성별 필터 (All)
        'ageBand': 'AGE_BAND_ALL',  # 연령 필터
        'period': 'REALTIME',  # 실시간
        'page': page,
        'size': size
    }

    # URL 파라미터 생성
    param_string = "&".join([f"{k}={v}" for k, v in params.items()])
    return f"{base_url}?{param_string}"

In [ ]:
import json
import requests

"""API를 통한 상품 ID 수집"""
url = build_api_url(1, 40)
ua = UserAgent()
session = requests.Session()
headers = {
    'User-Agent': ua.random,
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.musinsa.com/main/beauty/ranking',
    'Origin': 'https://www.musinsa.com',
}
session.headers.update(headers)

try:
    response = session.get(url, timeout=10)
    response.raise_for_status()

    data = response.json()
    print(f"페이지 API 응답 성공")
except requests.exceptions.RequestException as e:
   print(f"페이지 API 요청 실패: {e}")
except json.JSONDecodeError as e:
    print(f"페이지 JSON 파싱 실패: {e}")

product_ids = []
def recurse(obj):
    if isinstance(obj, dict):
        if obj.get('type') == 'PRODUCT_COLUMN' and 'id' in obj:
            product_ids.append(obj['id'])
        for v in obj.values():
            recurse(v)
    elif isinstance(obj, list):
        for item in obj:
            recurse(item)

recurse(data)
product_ids = list(dict.fromkeys(product_ids))  # 중복 제거

In [ ]:
# 리뷰 저장

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import datetime
import os

def get_review_html(driver, product_id):
    review_url = f"https://www.musinsa.com/review/goods/{product_id}?gf=A"
    driver.get(review_url)
    time.sleep(3)
    return driver.page_source

# 크롬 옵션 설정
options = Options()
# options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

# 드라이버 실행
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.musinsa.com/main/beauty/ranking?storeCode=beauty&sectionId=231&contentsId=&categoryCode=104001&gf=A"
driver.get(url)
time.sleep(3)

# datetime 폴더 생성
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = f"musinsa_reviews_{now}"
os.makedirs(save_dir, exist_ok=True)

# review 저장
for product_id in product_ids:
    html = get_review_html(driver, product_id)
    file_path = os.path.join(save_dir, f"{product_id}.html")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html)
    print(f"product_id {product_id} : 후기 페이지 저장 완료 -> {file_path}")

driver.quit()

In [ ]:
import requests

from fake_useragent import UserAgent

max_pages = 50
page_size = 20

session = requests.Session()
ua = UserAgent()
headers = {
    'User-Agent': ua.random,
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.musinsa.com/main/beauty/ranking',
    'Origin': 'https://www.musinsa.com',
}
session.headers.update(headers)
print("세션 설정 완료")

base_url = "https://goods.musinsa.com/api2/review/v1/view/list"
all_reviews = {}

for goods_no in product_ids:
        reviews = []
        for page in range(max_pages):
            params = {
                "page": page,
                "pageSize": page_size,
                "goodsNo": goods_no,
                "sort": "up_cnt_desc",
                "selectedSimilarNo": goods_no,
                "myFilter": "false",
                "hasPhoto": "false",
                "isExperience": "false"
            }
            param_string = "&".join([f"{k}={v}" for k, v in params.items()])
            url = f"{base_url}?{param_string}"
            try:
                response = session.get(url=url, timeout=10)
                response.raise_for_status()
                data = response.json()
                review_list = data.get("data", {}).get("list", [])
                if not review_list:
                    break  # 더 이상 리뷰가 없으면 중단
                reviews.extend(review_list)
            except requests.exceptions.RequestException as e:
                print(f"상품 {goods_no}의 {page}페이지 요청 실패")
                break
            except json.JSONDecodeError as e:
                print(f"페이지 {page} JSON 파싱 실패: {e}")
                break
            time.sleep(1)  # 서버 부하 방지용 딜레이
        all_reviews[goods_no] = reviews